This is the place where I experimented with things before commiting them to the package's code.

It's also where I put usage code, before I transform those to docs and/or tests.

# get_raw_data

In [3]:
from saying.util import get_raw_data

get_raw_data()

Returning named sources...


{'micheleriva_5421', 'x16bkkamz6rkb78rzt7op_75968'}

In [32]:
d = get_raw_data('micheleriva_5421')
len(d)

5421

In [33]:
d[:3]

[{'quote': 'Genius is one percent inspiration and ninety-nine percent perspiration.',
  'author': 'Thomas Edison'},
 {'quote': 'You can observe a lot just by watching.', 'author': 'Yogi Berra'},
 {'quote': 'A house divided against itself cannot stand.',
  'author': 'Abraham Lincoln'}]

In [4]:
dd = get_raw_data('x16bkkamz6rkb78rzt7op_75968')
len(dd)

75968

In [43]:
dd[1], dd[2:4]

(['2', "'QUOTE'", "'AUTHOR'", "'GENRE\\r'"],
 [['3',
   "'Age is an issue of mind over matter. If you don't mind",
   "it doesn't matter.'",
   "'Mark Twain'",
   "'age\\r'"],
  ['4',
   "'Anyone who stops learning is old",
   "whether at twenty or eighty. Anyone who keeps learning stays young. The greatest thing in life is to keep your mind young.'",
   "'Henry Ford'",
   "'age\\r'"]])

In [59]:
def remove_surrounding_single_quotes(s):
    if s.startswith("'") and s.endswith("'"):
        return s[1:-1]
    return s

def clean_sring(s):
    s = remove_surrounding_single_quotes(s)
    if s.endswith("\\r"):
        s = s[:-2]
    s = s.strip()
    return s


def dict_of_rows_and_columns(rows, columns):
    for row in rows:
        yield dict(zip(columns, row))


columns = list(map(lambda x: remove_surrounding_single_quotes(x).lower(), dd[1]))
ddd = list(dict_of_rows_and_columns(dd[2:], columns=columns))

ddd = [
    {clean_sring(k): clean_sring(v) for k, v in row.items()} for row in ddd
]

ddd[4]


{'2': '7',
 'quote': "A diplomat is a man who always remembers a woman's birthday but never remembers her age.",
 'author': 'Robert Frost',
 'genre': 'age'}

In [57]:

'age\\r'[-2]

'\\'

In [1]:
import re

def extract_sql_data(text):
    # Find all INSERT INTO statements
    insert_statements = re.findall(r"INSERT INTO `[^`]+` VALUES \((.*?)\);", text, re.DOTALL)

    data = []
    for statement in insert_statements:
        # Split the statement into individual records
        records = statement.split('),(')
        for record in records:
            # Clean and split the record into fields
            fields = record.strip().split(',')
            # Clean up each field (remove surrounding quotes and escape sequences)
            cleaned_fields = [re.sub(r"(^' | '$)", '', field).replace("\\'", "'").strip() for field in fields]
            data.append(cleaned_fields)
    
    return data

from saying.util import extract_sql_data




In [11]:
from saying.util import extract_sql_data

# Example SQL text to be processed
from saying.util import extract_sql_data, extract_sql_table_rows

# Example SQL text to be processed
sql_text = """
INSERT INTO `quotes` VALUES 
(1,'EXTRACTED DATA WITH HTTP:\\\\THEWEBMINER.COM','','\\r'),
(2,'QUOTE','AUTHOR','GENRE\\r'),
(3,'Age is an issue of mind over matter. If you don\\'t mind, it doesn\\'t matter.','Mark Twain','age\\r'),
(4,'Anyone who stops learning is old, whether at twenty or eighty. Anyone who keeps learning stays young. The greatest thing in life is to keep your mind young.','Henry Ford','age\\r');
"""

sql_text = """
INSERT INTO `quotes` VALUES 
(1,'EXTRACTED DATA WITH HTTP:\\\\THEWEBMINER.COM','','\\r'),
(2,'QUOTE','AUTHOR','GENRE\\r'),
(3,'Age is an issue of mind over matter. If you don\\'t mind, it doesn\\'t matter.','Mark Twain','age\\r'),
(4,'Anyone who stops learning is old, whether at twenty or eighty. Anyone who keeps learning stays young. The greatest thing in life is to keep your mind young.','Henry Ford','age\\r')
"""

sql_text = """-- MySQL dump 10.13  Distrib 5.5.37, for debian-linux-gnu (i686)
...
INSERT INTO `quotes` VALUES (1,'EXTRACTED DATA WITH HTTP:\\THEWEBMINER.COM','','\r'),(2,'QUOTE','AUTHOR','GENRE\r')
"""

# Call the function with the example SQL text
# quotes_data = extract_sql_data(sql_text)

# # Print the extracted data
# print(quotes_data)

# it = extract_sql_table_rows(sql_text)

# next(it)


sql_text = """
INSERT INTO `quotes` VALUES 
(1, 'Life is what happens when you\'re busy making other plans.', 'John Lennon', 'life');
(2, 'Get busy living or get busy dying.', 'Stephen King', 'life');
(3, 'The only impossible journey is the one you never begin.', 'Tony Robbins', 'motivation');
"""
sql_text = """
INSERT INTO `quotes` VALUES 
(1, 'Life is what happens when you\'re busy making other plans.', 'John Lennon
"""

sql_text = """
/*!40000 ALTER TABLE `quotes` DISABLE KEYS */;
INSERT INTO `quotes` VALUES (1,'EXTRACTED DATA WITH HTTP:\\THEWEBMINER.COM','','\r'),
(2,'QUOTE','AUTHOR','GENRE\r'),
(3,'Age is an issue of mind over matter. If you don\'t mind, it doesn\'t matter.','Mark Twain','age\r');
"""


sql_text = """
/*!40000 ALTER TABLE `quotes` DISABLE KEYS */;
INSERT INTO `quotes` (col1,col2,col3) VALUES 
(1,'One', 1.0),
(2,'Two', 2.0),
(3,'Three', 3.0);
"""

# Test the function with the example SQL text
quotes_data = extract_sql_data(sql_text)

# Print the extracted data
print(quotes_data)

text = sql_text
import re
insert_statements = re.findall(
    (
        r"INSERT INTO "
        "`[^`]+"  # table name
        "`.+?"  # could be nothing, or colmns...
        "VALUES.+?\((.*?)\);"  # statement (rows)
    ),
    text,
    re.DOTALL,
)

AttributeError: module 'dol' has no attribute 'base'

In [10]:
import sqlparse

def parse_sql_dump(sql_text):
    # Parse the SQL dump
    parsed = sqlparse.parse(sql_text)

    table_name = None
    columns_list = []
    rows = []

    for statement in parsed:
        if statement.get_type() == 'CREATE':
            # Extract table name
            tokens = [token for token in statement.tokens if not token.is_whitespace]
            for i, token in enumerate(tokens):
                if token.value.upper() == 'TABLE':
                    table_name = tokens[i + 1].get_real_name()
                elif token.ttype is sqlparse.tokens.Punctuation and token.value == '(':
                    # Extract columns
                    columns_list = [str(column).strip() for column in tokens[i + 1].get_identifiers()]
                    break

        elif statement.get_type() == 'INSERT':
            # Extract rows
            for token in statement.flatten():
                if token.value.upper().startswith('VALUES'):
                    values_str = token.value[6:].strip()
                    rows_str = values_str[1:-1]  # Remove surrounding parentheses
                    rows = [tuple(val.strip() for val in row.split(',')) for row in rows_str.split('),(')]

    return table_name, columns_list, rows

# Example usage
sql_text = """CREATE TABLE `quotes` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `quote` varchar(800) DEFAULT NULL,
  `author` varchar(100) DEFAULT NULL,
  `genre` varchar(100) DEFAULT NULL,
  PRIMARY KEY (`id`)
);
INSERT INTO `quotes` VALUES (1, 'Life is what happens...', 'John Lennon', 'life'), (2, 'Get busy living...', 'Stephen King', 'life');
"""

table_name, columns, rows = parse_sql_dump(sql_text)
print("Table:", table_name)
print("Columns:", columns)
print("Rows:", rows)


Table: quotes
Columns: []
Rows: [('',)]


In [23]:
from saying.util import graze, graze_urls


b = graze(graze_urls['x16bkkamz6rkb78rzt7op_75968'])
print(f"{len(b)=}")

# table_name, columns, rows = parse_sql_dump(b.decode())
print("Table:", table_name)
print("Columns:", columns)
print(f"{len(rows)=}")

len(b)=13733779
Table: quotes
Columns: []
len(rows)=1


In [26]:
rows

[('',)]

In [38]:
from saying.util import hash_text

t = set(map(lambda x: hash_text(x['quote']), d))
tt = set(map(lambda x: hash_text(x['text']), dd))
len(t & tt)

1485

In [10]:
from dol import ValueCodecs, wrap_kvs
from dol.trans import ValueCodec

d = {}
dd = ValueCodecs.json(indent=True)(d)
dd['a'] = {'b': 2}
d


# ValueCodecs.default.json
# import json
# json.dumps


{'a': '{\n "b": 2\n}'}

[{'quote': 'Genius is one percent inspiration and ninety-nine percent perspiration.',
  'author': 'Thomas Edison'},
 {'quote': 'You can observe a lot just by watching.', 'author': 'Yogi Berra'},
 {'quote': 'A house divided against itself cannot stand.',
  'author': 'Abraham Lincoln'}]

In [27]:
dd = get_raw_data('micheleriva')
len(dd)

5421

In [24]:
from hashlib import md5
import re

non_alphanumeric_re = re.compile(r'\W+')

def lower_alphanumeric(text):
    return non_alphanumeric_re.sub(' ', text).strip().lower()

def hash_text(text):
    """Return a hash of the text, ignoring punctuation and capitalization.
    
    >>> (assert hash_text('Hello, world!') 
    ...     ==  hash_text('hello world') 
    ...     == '5eb63bbbe01eeed093cb22bb8f5acdc3'
    ... )

    """
    normalized_text = lower_alphanumeric(text)
    return md5(normalized_text.encode()).hexdigest()




'5eb63bbbe01eeed093cb22bb8f5acdc3'